# CBOR vs JSON

Anton Antonov   
September 2024

-------

## Setup

In [2]:
use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use Data::Importers;
use Data::Reshapers;
use Data::Summarizers;

use XDG::BaseDirectory :terms;

use JSON::Fast;
use CBOR::Simple;

use NativeCall;

------

## Vector databases

Skim vector databases from the default directory and show summaries:

In [6]:
#% html
my @field-names = <id name item-count dimension version llm-service llm-embedding-model created>;
vector-database-objects(f=>'hash', :flat)
==> { $_.map({ $_<created> = $_<file>.IO.created.DateTime.Str.subst('T',' ').substr(^19); $_}).sort(*<created>).reverse }()
==> to-html(:@field-names)

id name item-count dimension version llm-service llm-embedding-model created a840beb1-b0fa-49af-b206-25f4644daae0 EconomicsAI 1074 768 0 gemini embedding-001 2025-04-14 00:39:39 e8042e57-9fea-43fe-b603-b4f4a31a67e1 FSMComands 35 1536 0 chatgpt text-embedding-3-small 2024-10-02 03:16:43 6877aa03-fb10-44a0-8a44-37121428cbbd words 20 768 0 gemini embedding-001 2024-09-21 13:36:37 f00209fa-0f35-41bf-8597-744dae9aab5d No833 283 4096 0 llama text-embedding-3-small 2024-09-19 01:31:12 a6b12f80-0ed7-4125-9e2b-9f77c11a16f8 No833 441 4096 0 llama text-embedding-3-small 2024-09-19 01:26:58 32bd53cb-7171-4d26-9a4f-11de377d3af1 words 20 4096 0 llama text-embedding-3-small 2024-09-19 01:14:27 5097c865-7fad-43d9-b7ca-456220d754b7 words 20 768 0 gemini embedding-001 2024-09-18 14:08:44 045f467c-193f-4df6-bec3-790d6c83ca64 No833 441 768 0 gemini embedding-001 2024-09-16 22:21:25 d2effebc-2cef-4b2b-84ca-5dcfa3c1864b No747 283 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:30 5cb40fbb-9f69-48ca-9fc1-03ec8059ed99 No747 283 768 0 gemini embedding-001 2024-09-12 13:32:23 44f19858-730e-4b96-86b7-81e701f9df8f No747 283 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:18 266b20ca-d917-4ac0-9b0a-7c420625666c No833 441 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:11

-----

## Import from JSON format

Get specific databases:

In [3]:
create-vector-database(file => 'a6b12f80-0ed7-4125-9e2b-9f77c11a16f8')

VectorDatabase(:id("a6b12f80-0ed7-4125-9e2b-9f77c11a16f8"), :name("No833"), :elements(441), :sources(442), :precision(num64))

In [ ]:
my @vdbs = vector-database-objects(f=>'hash', :flat).grep({ $_<id> ∈ <266b20ca-d917-4ac0-9b0a-7c420625666c d2effebc-2cef-4b2b-84ca-5dcfa3c1864b> }).map({ create-vector-database(file => $_<file>) })

-----

## Export to CBOR format

In [ ]:
@vdbs.head.export(format => 'cbor')

In [ ]:
my $vdbObj = LLM::RetrievalAugmentedGeneration::VectorDatabase.new();
$vdbObj.import(data-home.Str ~ '/raku/LLM/SemanticSearchIndex/SemSe-266b20ca-d917-4ac0-9b0a-7c420625666c.cbor')

In [ ]:
my $vdbObj2 = LLM::RetrievalAugmentedGeneration::VectorDatabase.new(num-type => 'num32');
$vdbObj2.import(data-home.Str ~ '/raku/LLM/SemanticSearchIndex/SemSe-266b20ca-d917-4ac0-9b0a-7c420625666c.json')

In [ ]:
$vdbObj2.vectors

In [ ]:
my %vectors = $vdbObj2.vectors;
%vectors .= map({ $_.key => CArray[num32].new($_.value) });

In [ ]:
my $vdbObj3 = LLM::RetrievalAugmentedGeneration::VectorDatabase.new(num-type => 'num32', :%vectors, items => $vdbObj2.items);

In [ ]:
$vdbObj3.export(format => 'cbor')

In [ ]:
my $dirname = LLM::RetrievalAugmentedGeneration::default-location();
my $vdbObj4 = create-vector-database(file => $dirname ~ "/SemSe-77da9a57-b53f-4b93-9935-ffa5476b8d7b.cbor")